In [2]:
import nltk

In [3]:
nltk.data.show_cfg('grammars/book_grammars/sql0.fcfg')

% start S
S[SEM=(?np + WHERE + ?vp)] -> NP[SEM=?np] VP[SEM=?vp]
VP[SEM=(?v + ?pp)] -> IV[SEM=?v] PP[SEM=?pp]
VP[SEM=(?v + ?ap)] -> IV[SEM=?v] AP[SEM=?ap]
NP[SEM=(?det + ?n)] -> Det[SEM=?det] N[SEM=?n]
PP[SEM=(?p + ?np)] -> P[SEM=?p] NP[SEM=?np]
AP[SEM=?pp] -> A[SEM=?a] PP[SEM=?pp]
NP[SEM='Country="greece"'] -> 'Greece'
NP[SEM='Country="china"'] -> 'China'
Det[SEM='SELECT'] -> 'Which' | 'What'
N[SEM='City FROM city_table'] -> 'cities'
IV[SEM=''] -> 'are'
A[SEM=''] -> 'located'
P[SEM=''] -> 'in'


In [4]:
from nltk import load_parser
cp = load_parser('grammars/book_grammars/sql0.fcfg')
query = 'What cities are located in China'
trees = list(cp.parse(query.split()))
answer = trees[0].label()['SEM']
answer = [s for s in answer if s]
q = ' '.join(answer)
print(q)

SELECT City FROM city_table WHERE Country="china"


In [5]:
from nltk.sem import chat80
rows = chat80.sql_query('corpora/city_database/city.db', q)
for r in rows: print(r[0], end=" ")

canton chungking dairen harbin kowloon mukden peking shanghai sian tientsin 

In [6]:
nltk.boolean_ops()

negation       	-
conjunction    	&
disjunction    	|
implication    	->
equivalence    	<->


In [7]:
read_expr = nltk.sem.Expression.fromstring
read_expr('-(P & Q)')

<NegatedExpression -(P & Q)>

In [8]:
read_expr('P & Q')

<AndExpression (P & Q)>

In [9]:
read_expr('P | (R -> Q)')

<OrExpression (P | (R -> Q))>

In [10]:
read_expr('P <-> -- P')

<IffExpression (P <-> --P)>

In [83]:
# lp = nltk.sem.Expression.fromstring
# SnF = read_expr('SnF')
# NotFnS = read_expr('-FnS')
# R = read_expr('SnF -> -FnS')
# prover = nltk.Prover9()
# prover.prove(NotFnS, [SnF, R])

In [84]:
val = nltk.Valuation([('P', True), ('Q', True), ('R', False)])

In [85]:
val['P']

True

In [86]:
dom = set()
g = nltk.Assignment(dom)

In [87]:
m = nltk.Model(dom, val)

In [88]:
print(m.evaluate('(P & Q)', g))

True


In [89]:
print(m.evaluate('-(P & Q)', g))

False


In [90]:
print(m.evaluate('(P & R)', g))

False


In [91]:
print(m.evaluate('(P | R)', g))

True


In [92]:
read_expr = nltk.sem.Expression.fromstring
expr = read_expr('walk(angus)', type_check=True)
expr.argument

<ConstantExpression angus>

In [93]:
expr.argument.type

e

In [94]:
expr.function

<ConstantExpression walk>

In [95]:
expr.function.type

<e,?>

In [96]:
sig = {'walk': '<e, t>'}
expr = read_expr('walk(angus)', signature=sig)
expr.function.type

e

In [97]:
read_expr = nltk.sem.Expression.fromstring
read_expr('dog(cyril)').free()

set()

In [98]:
read_expr('dog(x)').free()

{Variable('x')}

In [99]:
read_expr('own(angus, cyril)').free()

set()

In [100]:
read_expr('exists x.dog(x)').free()

set()

In [101]:
read_expr('((some x. walk(x)) -> sing(x))').free()

{Variable('x')}

In [102]:
read_expr('exists x.own(y, x)').free()

{Variable('y')}

In [103]:
# NotFnS = read_expr('-north_of(f, s)')
# SnF = read_expr('north_of(s, f)')
# R = read_expr('all x. all y. (north_of(x, y) -> -north_of(y, x))')
# prover = nltk.Prover9()
# prover.prove(NotFnS, [SnF, R])

In [104]:
# FnS = read_expr('north_of(f, s)')
# prover.prove(FnS, [SnF, R])

In [105]:
dom = {'b', 'o', 'c'}

In [106]:
v = """
bertie => b
olive => o
cyril => c
boy => {b}
girl => {o}
dog => {c}
walk => {o, c}
see => {(b, o), (c, b), (o, c)}
"""
val = nltk.Valuation.fromstring(v)
print(val)

{'bertie': 'b',
 'boy': {('b',)},
 'cyril': 'c',
 'dog': {('c',)},
 'girl': {('o',)},
 'olive': 'o',
 'see': {('o', 'c'), ('c', 'b'), ('b', 'o')},
 'walk': {('o',), ('c',)}}


In [107]:
('o', 'c') in val['see']

True

In [108]:
('b',) in val['boy']

True

In [109]:
g = nltk.Assignment(dom, [('x', 'o'), ('y', 'c')])
g

{'x': 'o', 'y': 'c'}

In [110]:
print(g)

g[c/y][o/x]


In [111]:
m = nltk.Model(dom, val)
m.evaluate('see(olive, y)', g)

True

In [112]:
g['y']

'c'

In [113]:
m.evaluate('see(y, x)', g)

False

In [114]:
g.purge()
g

{}

In [115]:
m.evaluate('see(olive, y)', g)

'Undefined'

In [116]:
m.evaluate('see(bertie, olive) & boy(bertie) & -walk(bertie)', g)

True

In [117]:
m.evaluate('exists x.(girl(x) & walk(x))', g)

True

In [118]:
m.evaluate('girl(x) & walk(x)', g.add('x', 'o'))

True

In [119]:
fmla1 = read_expr('girl(x) | boy(x)')
m.satisfiers(fmla1, 'x', g)

{'b', 'o'}

In [120]:
fmla2 = read_expr('girl(x) -> walk(x)')
m.satisfiers(fmla2, 'x', g)

{'b', 'c', 'o'}

In [121]:
fmla3 = read_expr('walk(x) -> girl(x)')
m.satisfiers(fmla3, 'x', g)

{'b', 'o'}

In [122]:
m.evaluate('all x.(girl(x) -> walk(x))', g)

True

In [123]:
v2 = """
bruce => b
elspeth => e
julia => j
matthew => m
person => {b, e, j, m}
admire => {(j, b), (b, b), (m, e), (e, m)}
"""

In [124]:
val2 = nltk.Valuation.fromstring(v2)

In [125]:
dom2 = val2.domain
m2 = nltk.Model(dom2, val2)
g2 = nltk.Assignment(dom2)
fmla4 = read_expr('(person(x) -> exists y.(person(y) & admire(x, y)))')
m2.satisfiers(fmla4, 'x', g2)

{'b', 'e', 'j', 'm'}

In [126]:
fmla5 = read_expr('(person(y) & all x.(person(x) -> admire(x, y)))')
m2.satisfiers(fmla5, 'y', g2)

set()

In [127]:
fmla6 = read_expr('(person(y) & all x.((x = bruce | x = julia) -> admire(x, y)))')
m2.satisfiers(fmla6, 'y', g2)

{'b'}

In [137]:
a3 = read_expr('exists x.(man(x) & walks(x))')
c1 = read_expr('mortal(socrates)')
c2 = read_expr('-mortal(socrates)')
mb = nltk.Mace(5)
print(mb.build_model(None, [a3, c1]))

LookupError: 

===========================================================================
NLTK was unable to find the mace4 file!
Use software specific configuration parameters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on mace4, see:
    <https://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [129]:
print(mb.build_model(None, [a3, c2]))

In [131]:
print(mb.build_model(None, [c1, c2]))

In [133]:
a4 = read_expr('exists y. (woman(y) & all x. (man(x) -> love(x,y)))')
a5 = read_expr('man(adam)')
a6 = read_expr('woman(eve)')
g = read_expr('love(adam,eve)')
mc = nltk.MaceCommand(g, assumptions=[a4, a5, a6])
mc.build_model()

In [135]:
print(mc.valuation)

In [136]:
a7 = read_expr('all x. (man(x) -> -woman(x))')
g = read_expr('love(adam,eve)')
mc = nltk.MaceCommand(g, assumptions=[a4, a5, a6, a7])
mc.build_model()

LookupError: 

===========================================================================
NLTK was unable to find the mace4 file!
Use software specific configuration parameters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on mace4, see:
    <https://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [ ]:
print(mc.valuation)

In [ ]:
read_expr = nltk.sem.Expression.fromstring
expr = read_expr(r'\x.(walk(x) & chew_gum(x))')
expr

In [ ]:
expr.free()

In [ ]:
print(read_expr(r'\x.(walk(x) & chew_gum(y))'))

In [ ]:
expr = read_expr(r'\x.(walk(x) & chew_gum(x))(gerald)')
print(expr)

In [ ]:
print(expr.simplify())

In [ ]:
print(read_expr(r'\x.\y.(dog(x) & own(y, x))(cyril)').simplify())

In [ ]:
print(read_expr(r'\x y.(dog(x) & own(y, x))(cyril, angus)').simplify())

In [ ]:
expr1 = read_expr('exists x.P(x)')
print(expr1)

In [ ]:
expr2 = expr1.alpha_convert(nltk.sem.Variable('z'))
print(expr2)

In [ ]:
expr1 == expr2

In [ ]:
expr3 = read_expr('\P.(exists x.P(x))(\y.see(y, x))')
print(expr3)

In [ ]:
print(expr3.simplify())

In [ ]:
read_expr = nltk.sem.Expression.fromstring
tvp = read_expr(r'\X x.X(\y.chase(x,y))')
np = read_expr(r'(\P.exists x.(dog(x) & P(x)))')
vp = nltk.sem.ApplicationExpression(tvp, np)
print(vp)

In [ ]:
print(vp.simplify())

In [ ]:
from nltk import load_parser
parser = load_parser('grammars/book_grammars/simple-sem.fcfg', trace=0)
sentence = 'Angus gives a bone to every dog'
tokens = sentence.split()
for tree in parser.parse(tokens):
    print(tree.label()['SEM'])

In [ ]:
sents = ['Irene walks', 'Cyril bites an ankle']
grammar_file = 'grammars/book_grammars/simple-sem.fcfg'
for results in nltk.interpret_sents(sents, grammar_file):
    for (synrep, semrep) in results:
        print(synrep)

In [ ]:
v = """
bertie => b
olive => o
cyril => c
boy => {b}
girl => {o}
dog => {c}
walk => {o, c}
see => {(b, o), (c, b), (o, c)}
"""
val = nltk.Valuation.fromstring(v)
g = nltk.Assignment(val.domain)
m = nltk.Model(val.domain, val)
sent = 'Cyril sees every boy'
grammar_file = 'grammars/book_grammars/simple-sem.fcfg'
results = nltk.evaluate_sents([sent], grammar_file, m, g)[0]
for (syntree, semrep, value) in results:
    print(semrep)
    print(value)

In [138]:
from nltk.sem import cooper_storage as cs
sentence = 'every girl chases a dog'
trees = cs.parse_with_bindops(sentence, grammar='grammars/book_grammars/storage.fcfg')
semrep = trees[0].label()['SEM']
cs_semrep = cs.CooperStore(semrep)
print(cs_semrep.core)

chase(z2,z4)


In [139]:
for bo in cs_semrep.store:
    print(bo)

bo(\P.all x.(girl(x) -> P(x)),z2)
bo(\P.exists x.(dog(x) & P(x)),z4)


In [140]:
cs_semrep.s_retrieve(trace=True)

Permutation 1
   (\P.all x.(girl(x) -> P(x)))(\z2.chase(z2,z4))
   (\P.exists x.(dog(x) & P(x)))(\z4.all x.(girl(x) -> chase(x,z4)))
Permutation 2
   (\P.exists x.(dog(x) & P(x)))(\z4.chase(z2,z4))
   (\P.all x.(girl(x) -> P(x)))(\z2.exists x.(dog(x) & chase(z2,x)))


In [141]:
for reading in cs_semrep.readings:
    print(reading)

exists x.(dog(x) & all z3.(girl(z3) -> chase(z3,x)))
all x.(girl(x) -> exists z4.(dog(z4) & chase(x,z4)))


In [142]:
read_dexpr = nltk.sem.DrtExpression.fromstring
drs1 = read_dexpr('([x, y], [angus(x), dog(y), own(x, y)])')
print(drs1)

([x,y],[angus(x), dog(y), own(x,y)])


In [143]:
drs1.draw()

In [144]:
print(drs1.fol())

exists x y.(angus(x) & dog(y) & own(x,y))


In [145]:
drs2 = read_dexpr('([x], [walk(x)]) + ([y], [run(y)])')
print(drs2)

(([x],[walk(x)]) + ([y],[run(y)]))


In [146]:
print(drs2.simplify())

([x,y],[walk(x), run(y)])


In [147]:
drs3 = read_dexpr('([], [(([x], [dog(x)]) -> ([y],[ankle(y), bite(x, y)]))])')
print(drs3.fol())

all x.(dog(x) -> exists y.(ankle(y) & bite(x,y)))


In [148]:
drs4 = read_dexpr('([x, y], [angus(x), dog(y), own(x, y)])')
drs5 = read_dexpr('([u, z], [PRO(u), irene(z), bite(u, z)])')
drs6 = drs4 + drs5
print(drs6.simplify())

([u,x,y,z],[angus(x), dog(y), own(x,y), PRO(u), irene(z), bite(u,z)])


In [149]:
print(drs6.simplify().resolve_anaphora())

([u,x,y,z],[angus(x), dog(y), own(x,y), (u = [x,y,z]), irene(z), bite(u,z)])


In [150]:
from nltk import load_parser
parser = load_parser('grammars/book_grammars/drt.fcfg', logic_parser=nltk.sem.drt.DrtParser())
trees = list(parser.parse('Angus owns a dog'.split()))
print(trees[0].label()['SEM'].simplify())

([x,z6],[Angus(x), dog(z6), own(x,z6)])


In [151]:
dt = nltk.DiscourseTester(['A student dances', 'Every student is a person'])
dt.readings()

LookupError: 

===========================================================================
NLTK was unable to find the mace4 file!
Use software specific configuration parameters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on mace4, see:
    <https://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [ ]:
dt.add_sentence('No person dances', consistchk=True)

In [ ]:
dt.retract_sentence('No person dances', verbose=True)

In [ ]:
dt.add_sentence('A person dances', informchk=True)

In [ ]:
from nltk.tag import RegexpTagger
tagger = RegexpTagger(
    [('^(chases|runs)$', 'VB'),
     ('^(a)$', 'ex_quant'),
     ('^(every)$', 'univ_quant'),
     ('^(dog|boy)$', 'NN'),
     ('^(He)$', 'PRP')
])
rc = nltk.DrtGlueReadingCommand(depparser=nltk.MaltParser(tagger=tagger))
dt = nltk.DiscourseTester(['Every dog chases a boy', 'He runs'], rc)
dt.readings()

In [ ]:
dt.readings(show_thread_readings=True)

In [ ]:
dt.readings(show_thread_readings=True, filter=True)